---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [4]:
S = 50                      # initial price of underlying
K = 50                      # strike price
r = 0.01                    # risk-free rate per period
q = 0.02                    # dividend yield
sigma = 0.30                # volatility per year
T = 1                       # years to maturity
n = 10                      # number of periods
kind = 'European call'      # type of option
# kind options: 'European call', 'European put', 'American call', 'American put',

# FUNCTIONS

In [5]:
import numpy as np
import plotly.graph_objects as go


def europeanTree(S, K, r, q, sigma, T, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    # parameters
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)    

    # build tree
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x = disc * (p * x[:-1] + (1 - p) * x[1:])
        lst.insert(0, x)
    return [list(x) for x in lst], p


def americanTree(S, K, r, q, sigma, T, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    # parameters
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)   

    # build tree
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x0 = disc * (p * x[:-1] + (1 - p) * x[1:])
        t = len(x0) - 1
        St = [S * u ** (t - 2 * i) for i in range(t + 1)]
        x = np.maximum(x0, f(St))
        lst.insert(0, x)
    return [list(x) for x in lst], p

# CALCULATIONS

In [6]:
# Stock price tree
dt = T/n
u  = np.exp(sigma * np.sqrt(dt))
stock_tree = [[S * u ** (t - 2 * i) for i in range(t + 1)] for t in range(n + 1)]

# Option tree
kind_split = kind.split(" ")
Tree = europeanTree if kind_split[0] == "European" else americanTree
option_tree, prob = Tree(S, K, r, q, sigma, T, n, kind_split[1])
option_value = option_tree[0][0]

# Summarize output
print(f'{kind} value at date 0:\t\t${option_value:.2f}')
print(f'Risk-neutral probability of up state:\t{prob:.1%}')

European call value at date 0:		$5.49
Risk-neutral probability of up state:	47.1%


# FIGURE: UNDERLYING PRICE

In [7]:
tree = stock_tree

# Prep data for plotting
spliced = []
for a, b in zip(tree[1:], tree[:-1]):
    x = []
    for i in range(len(a)):
        x.append(a[i])
        try:
            x.append(b[i])
        except:
            pass
    spliced.append(x)


# Make figure
fig = go.Figure()
string = "$%{y:,.2f}<extra></extra>"
for i in range(len(tree) - 1):
    x = [1, 0, 1]
    for j in range(i):
        x.append(0)
        x.append(1)
    x = np.array(x) + i
    y = spliced[i]
    trace = go.Scatter(
        x=x,
        y=y,
        mode="lines+markers",
        hovertemplate=string,
        marker=dict(size=12, color='blue'),
        line=dict(color='blue'),
        showlegend=False
    )
    fig.add_trace(trace)

fig.update_layout(
    xaxis=dict(tickmode="linear", tick0=0, dtick=1, title="Time"),
    yaxis=dict(tickprefix="$", tickformat=",.0f", title="Underlying Price"),
    template="plotly_white"
)

# FIGURE: OPTION

In [8]:
tree = option_tree

# Prep data for plotting
spliced = []
for a, b in zip(tree[1:], tree[:-1]):
    x = []
    for i in range(len(a)):
        x.append(a[i])
        try:
            x.append(b[i])
        except:
            pass
    spliced.append(x)
spliced

# Make figure
fig = go.Figure()
string = "$%{y:,.2f}<extra></extra>"
for i in range(len(tree) - 1):
    x = [1, 0, 1]
    for j in range(i):
        x.append(0)
        x.append(1)
    x = np.array(x) + i
    y = spliced[i]
    trace = go.Scatter(
        x=x,
        y=y,
        mode="lines+markers",
        hovertemplate=string,
        marker=dict(size=12, color='green'),
        line=dict(color='green'),
        showlegend=False
    )
    fig.add_trace(trace)

fig.update_layout(
    xaxis=dict(
        tickmode="linear", 
        tick0=0, 
        dtick=1, 
        title="Time"
    ),
    yaxis=dict(
        tickprefix="$", 
        tickformat=",.0f", 
        title=kind.title() + " Value", 
        autorange="reversed" if kind_split[1]=="put" else None
    ),
    template="plotly_white"
)

fig.show()